In [2]:
from urllib.request import urlopen
import urllib
import time
import http.cookiejar as cookiejar
import pandas as pd
import numpy as np
import os.path
import bs4 as bs
import re
import json
import os
from matplotlib import pyplot as plt
from matplotlib import style
from scipy import stats

style.use('seaborn-deep')

import pickle

from datalab_beta import *


hello


In [3]:
web_address = 'https://www.fifaindex.com/teams/fifa18/'

df = pd.DataFrame()

for day in range(1,260,1):
    for pag in range(1,30):
    
        source_address = web_address + '05_' + str(day) + '/' + str(pag) + '/' + '?type=1'
        print('Day:', str(day))
        print(pag)

        try:
            soup = get_soup(source_address)
            result_list =  soup.find('div', {'id': 'no-more-tables'})
        except:
            print('Page found.')
            break

        date = str(soup.find('ol', {'class': 'breadcrumb'}))

        if df.empty:
            df = pd.read_html(str(result_list))[0]
            df['date'] = date
        else:
            temp_df = pd.read_html(str(result_list))[0]
            temp_df['date'] = date
            df = df.append(temp_df)

Day: 1
1
Could not create soup.
Could not create soup.
Page found.
Day: 2
1
Could not create soup.
Could not create soup.
Page found.
Day: 3
1
Could not create soup.
Could not create soup.
Page found.
Day: 4
1
Could not create soup.
Could not create soup.
Page found.
Day: 5
1
Could not create soup.
Could not create soup.
Page found.
Day: 6
1
Could not create soup.
Could not create soup.
Page found.
Day: 7
1
Could not create soup.
Could not create soup.
Page found.
Day: 8
1
Could not create soup.
Could not create soup.
Page found.
Day: 9
1
Could not create soup.
Could not create soup.
Page found.
Day: 10
1
Could not create soup.
Could not create soup.
Page found.
Day: 11
1
Could not create soup.
Could not create soup.
Page found.
Day: 12
1
Could not create soup.
Could not create soup.
Page found.
Day: 13
1
Could not create soup.
Could not create soup.
Page found.
Day: 14
1
Could not create soup.
Could not create soup.
Page found.
Day: 15
1
Could not create soup.
Could not create soup.
P

In [4]:
df.head()

""


In [ ]:
df.columns = ['mid', 'ovr', 'att', 'def', 'league', '1', '2', '3', 'name', 'date']
df = df[['date', 'name', 'att', 'def', 'mid', 'ovr']]

In [ ]:
len(df.dropna())

In [ ]:
df = df.dropna()

In [ ]:
df.iloc[0].date

In [ ]:
def split_date(s):
    date = s.split('href="/teams/')[-1].split('>')[1].split('<')[0].split(',')[1]
    return date

In [7]:
df['date'] = df.date.apply(split_date)

AttributeError: 'DataFrame' object has no attribute 'date'

In [ ]:
df.head()

In [ ]:
df['date'] = df.date.apply(text_norm)
df['name'] = df.name.apply(text_norm)
df = df.reset_index(drop=True)
df = df.drop_duplicates()
to_pickle(df, 'team_stats')

In [5]:
df = read_pickle('team_stats.pickle')

In [6]:
df.head()

,date,name,att,def,mid,ovr
0,2004,france,94,84,89,88
1,2004,brazil,92,87,88,88
2,2004,spain,90,86,88,88
3,2004,england,89,88,88,88
4,2004,italy,92,86,83,87


In [7]:
df = df.groupby(['name', 'date']).mean()
df = df.reset_index()
df.index = pd.DatetimeIndex(df.date).year
df = df.drop(labels='date', axis=1)

In [8]:
df.head()

,name,att,def,mid,ovr
date,,,,,
2004,argentina,86.0,87.0,85.0,85.0
2005,argentina,87.0,85.0,81.0,83.0
2006,argentina,87.0,78.0,80.0,83.0
2007,argentina,85.0,80.0,80.0,83.0
2008,argentina,87.0,82.0,84.0,84.0


In [9]:
df = df.reset_index()
year_dic = {}

for year in df.date.unique():
    df_year = df[df.date == year]
    df_year = df_year[df_year.ovr == df_year.ovr.max()]
    year_dic[year] = [max(df_year.ovr.tolist()), df_year.name.tolist()]

In [10]:
mean_ovrs = []
for year in df.date.unique():
    df_year = df[df.date == year]
    mean_ovrs.append(df_year.ovr.mean())

In [11]:
years = list(year_dic.keys())
max_ovrs = [i[0] for i in list(year_dic.values())]
teams = [i[1] for i in list(year_dic.values())]

In [12]:
teams_str = []
for t in teams:
    try:
        teams_str.append(str(', '.join(t)))
    except:
        teams_str.append(t)

In [13]:
df.head()

,date,name,att,def,mid,ovr
0,2004,argentina,86.0,87.0,85.0,85.0
1,2005,argentina,87.0,85.0,81.0,83.0
2,2006,argentina,87.0,78.0,80.0,83.0
3,2007,argentina,85.0,80.0,80.0,83.0
4,2008,argentina,87.0,82.0,84.0,84.0


In [14]:
import plotly.plotly as py
import plotly
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='rodrigonader', api_key='MFs1Eh244N8I7ePAG3Va')

trace1= go.Bar(
            x=years,
            y=max_ovrs,
            text=teams_str,
            name = 'Max Performance',
            marker=dict(
            color='rgb(103,113,242)')
    )

trace2 = go.Scatter(
    x = years,
    y = mean_ovrs,
    mode = 'lines+markers',
    name = 'Mean Performance',
    marker = dict(
        size = 12,
        color = 'rgba(300, 300, 300, .9)',
        line = dict(
            width = 2)
        )
)


data = [trace1, trace2]

layout = go.Layout(
    title='Team performance by year',
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='text-hover-bar')

In [24]:
data=[]
for year in years:
    
    values = df.ovr[df.date==year].tolist()
    

    trace1 = go.Scatter(
                x=[year] * len(values),
                y=values,
                mode='markers',
                marker=dict(
                color='rgb(150,150,150)')
        )
    
    
    data.append(trace1)


fig = go.Figure(data=data)

py.iplot(fig, filename='text-hover-bar')

In [ ]:
# mais vizualizações aqui

In [15]:
results = pd.read_csv('match_results.csv')

In [16]:
results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [17]:
results = results.drop(['city', 'tournament', 'country'], axis=1)
results.home_team = results.home_team.apply(text_norm)
results.away_team = results.away_team.apply(text_norm)
results.index = pd.DatetimeIndex(results.date).year
results = results.drop('date', 1)
results.head()

,home_team,away_team,home_score,away_score,neutral
date,,,,,
1872,scotland,england,0,0,False
1873,england,scotland,4,2,False
1874,scotland,england,2,1,False
1875,england,scotland,2,2,False
1876,scotland,england,3,0,False


In [18]:
results = results.loc[2004:2017]
results.tail()

df_teams = list(df.name.unique())
results = results.reset_index()

for index, row in results.iterrows():
    if row.home_team not in df_teams:
        results.loc[index, 'home_team'] = None
    if row.away_team not in df_teams:
        results.loc[index, 'away_team'] = None
        
results = results.dropna()

In [19]:
results['att1'] = np.nan
results['def1'] = np.nan
results['mid1'] = np.nan
results['ovr1'] = np.nan
results['att2'] = np.nan
results['def2'] = np.nan
results['mid2'] = np.nan
results['ovr2'] = np.nan

for index, row in results.iterrows():
    date = row.date
    team1 = row.home_team
    team2 = row.away_team
    try:
        results.loc[index, 'att1'] = df[df.date == date][df.name == team1]['att'].iloc[0]
        results.loc[index, 'def1'] = df[df.date == date][df.name == team1]['def'].iloc[0]
        results.loc[index, 'mid1'] = df[df.date == date][df.name == team1]['mid'].iloc[0]
        results.loc[index, 'ovr1'] = df[df.date == date][df.name == team1]['ovr'].iloc[0]
        results.loc[index, 'att2'] = df[df.date == date][df.name == team2]['att'].iloc[0]
        results.loc[index, 'def2'] = df[df.date == date][df.name == team2]['def'].iloc[0]
        results.loc[index, 'mid2'] = df[df.date == date][df.name == team2]['mid'].iloc[0]
        results.loc[index, 'ovr2'] = df[df.date == date][df.name == team2]['ovr'].iloc[0]
        
    except:
        pass

C:\Users\user\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:15: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\user\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:16: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\user\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:17: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\user\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:18: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\user\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:19: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\user\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__

In [20]:
results = results.dropna()

In [21]:
results.head()

,date,home_team,away_team,home_score,away_score,neutral,att1,def1,mid1,ovr1,att2,def2,mid2,ovr2
31,2004,sweden,norway,0,3,True,83.0,75.0,76.0,77.0,76.0,77.0,73.0,75.0
73,2004,cameroon,nigeria,1,2,True,72.0,75.0,78.0,75.0,77.0,74.0,71.0,73.0
79,2004,tunisia,nigeria,1,1,False,69.0,63.0,68.0,64.0,77.0,74.0,71.0,73.0
93,2004,belgium,france,0,2,False,74.0,71.0,75.0,73.0,94.0,84.0,89.0,88.0
95,2004,croatia,germany,1,2,False,74.0,80.0,75.0,76.0,81.0,85.0,85.0,85.0


In [22]:
results['score'] = results.home_score - results.away_score
# results = results.drop(['home_score', 'away_score', 'home_team', 'away_team'], 1)

In [32]:
results['winner'] = None
results['winner'][results.score > 0] = 1
results['winner'][results.score < 0] = -1
results['winner'][results.score == 0] = 0

C:\Users\user\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\user\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\user\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [33]:
results['att'] = results['att1'] - results['att2']
results['def'] = results['def1'] - results['def2']
results['mid'] = results['mid1'] - results['mid2']
results['ovr'] = results['ovr1'] - results['ovr2']


In [34]:
results = results[results.winner != 0]

to_drop = results[results.winner == 1].sample(247)
results = results.drop(labels=to_drop.index, axis=0)

In [35]:
results.head()

,date,home_team,away_team,home_score,away_score,neutral,att1,def1,mid1,ovr1,att2,def2,mid2,ovr2,score,att,def,mid,ovr,winner
31,2004,sweden,norway,0,3,True,83.0,75.0,76.0,77.0,76.0,77.0,73.0,75.0,-3,7.0,-2.0,3.0,2.0,-1
73,2004,cameroon,nigeria,1,2,True,72.0,75.0,78.0,75.0,77.0,74.0,71.0,73.0,-1,-5.0,1.0,7.0,2.0,-1
93,2004,belgium,france,0,2,False,74.0,71.0,75.0,73.0,94.0,84.0,89.0,88.0,-2,-20.0,-13.0,-14.0,-15.0,-1
95,2004,croatia,germany,1,2,False,74.0,80.0,75.0,76.0,81.0,85.0,85.0,85.0,-1,-7.0,-5.0,-10.0,-9.0,-1
98,2004,greece,bulgaria,2,0,False,75.0,81.0,78.0,79.0,81.0,69.0,74.0,73.0,2,-6.0,12.0,4.0,6.0,1


In [36]:
trace = go.Scatter(
    x = results.ovr1,
    y = results.score,
    mode = 'markers',
    name = 'Mean Performance',
    marker = dict(
        size = 12,
        color = 'rgba(300, 300, 300, .9)',
        line = dict(
            width = 2)
        )
)


data = [trace]

fig = go.Figure(data=data)

py.iplot(fig, filename='text-hover-bar')

In [29]:
trace = go.Scatter3d(
    x = results.att,
    y = results.def1,
    z = results.mid1,
    mode = 'markers',
    name = 'Mean Performance',
    marker = dict(
        size = np.power(results.score.tolist(), 2),
        color = 'rgba(100, 150, 300, .9)',
        line = dict(
            width = 1)
        )
)


data = [trace]

layout = {
  "margin": {
    "r": 10, 
    "t": 25, 
    "b": 40, 
    "l": 60
  }, 
  "paper_bgcolor": "rgb(243, 243, 243)", 
  "plot_bgcolor": "rgb(243, 243, 243)", 
  "scene": {
    "xaxis": {
      "gridcolor": "rgb(255, 255, 255)", 
      "gridwidth": 2, 
      "ticklen": 5, 
      "title": "Attack", 
      "type": "log", 
      "zerolinewidth": 1
    }, 
    "yaxis": {
      "gridcolor": "rgb(255, 255, 255)", 
      "ticklen": 5, 
      "title": "Defense", 
      "zerolinewidth": 1
    }, 
    "zaxis": {
      "gridcolor": "rgb(255, 255, 255)", 
      "ticklen": 5, 
      "title": "Midside", 
      "type": "log", 
      "zerolinewidth": 1
    }
  }, 
  "title": "Teams Performance 3D", 
  "xaxis": {"domain": [0, 1]}, 
  "yaxis": {"domain": [0, 1]}
}
fig = go.Figure(data=data, layout=layout)

# fig = go.Figure(data=data)

py.iplot(fig, filename='text-hover-bar')

In [ ]:
# to_pickle(results, 'results_processed')

In [37]:
def prepare(data, target_col='price_area'):

	features = data.drop(target_col, 1)
	target = data[target_col]
	x = to_array(features)

	y = to_array(target)

	return x, y

In [38]:
results.winner = results.winner.apply(to_float)

results.replace(np.inf, np.nan)
results = results.dropna()

results = results.sample(frac=1)

data = results.drop(['date', 'score'], 1)
data = results[['def', 'ovr', 'att', 'mid', 'winner']]
train, test = train_test_split(data, 0.2)
train.reset_index(drop=True, inplace=True)
x_train, y_train = prepare(train, target_col='winner')
x_test, y_test = prepare(test, target_col='winner')

In [ ]:
This reveals that, althought it may seem that the overall is the consolidation of the other 3 attributes, this is not,
actually true, since, when we remove those other features, the regressor performs less accurate.

In [39]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

In [40]:
lr = LogisticRegression()
lr.fit(x_train, y_train)

accuracy_score(y_test, lr.predict(x_test))

0.73711340206185572

In [41]:
rf = RandomForestClassifier()
rf.fit(x_train, y_train)

accuracy_score(y_test, rf.predict(x_test))

0.66494845360824739

In [42]:
svc = SVC(kernel='linear')
svc.fit(x_train, y_train)

accuracy_score(y_test, svc.predict(x_test))

0.73195876288659789

In [43]:
test['prediction'] = svc.predict(x_test)

C:\Users\user\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [44]:
test.head()

,def,ovr,att,mid,winner,prediction
0,-8.52381,-6.047619,-1.698413,-4.777778,-1.0,-1.0
1,3.00000,0.000000,-2.000000,-1.000000,-1.0,-1.0
2,-2.50000,-0.500000,2.500000,0.000000,-1.0,-1.0
3,1.00000,4.000000,7.000000,5.000000,-1.0,1.0
4,-1.22500,0.475000,-1.250000,-0.175000,1.0,-1.0


In [46]:
first_score = accuracy_score(test.winner[test.winner > 0].values, test.prediction[test.winner > 0].values)
second_score = accuracy_score(test.winner[test.winner < 0].values, test.prediction[test.winner < 0].values)

In [49]:
trace1= go.Bar(
            x=['First Team Wins', 'Second Team Wins'],
            y=[first_score, second_score],
            name = 'Model Scores',
            marker=dict(
            color='rgb(50,300,160)'
            )
    
    )




data = [trace1]


# fig = go.Figure(data=data, layout=layout)

py.iplot(data, filename='Model-Scores')

In [50]:
web_address = 'https://www.fifaindex.com/teams/fifa18wc_262/'

wc = pd.DataFrame()

for pag in range(1,30):

    source_address = web_address + str(pag) + '/' + '?type=1'
    print(pag)

    try:
        soup = get_soup(source_address)
        result_list =  soup.find('div', {'id': 'no-more-tables'})
    except:
        print('Page not found.')
        break

    if df.empty:
        wc = pd.read_html(str(result_list))[0]
        wc['date'] = date
    else:
        temp_wc = pd.read_html(str(result_list))[0]
        temp_wc['date'] = date
        wc = wc.append(temp_wc)

1


ValueError: No tables found

In [ ]:
wc.columns = ['0', 'name', '1', 'att', 'mid', 'def', 'ovr', '2', '3', '4']
wc = wc[['name', 'att', 'def', 'mid', 'ovr']]

In [ ]:
wc.name = wc.name.apply(text_norm)

In [51]:
wc = read_pickle('world_cup_teams.pickle')
wc.head()

,name,att,def,mid,ovr
0,brazil,87,84,86,86
1,germany,85,85,85,86
2,france,86,81,84,85
3,spain,84,85,86,85
4,portugal,88,82,83,84


In [52]:
def match(wc, team1, team2, model):
    
    match = pd.DataFrame(columns=['att1','def1','mid1','ovr1','att2','def2','mid2','ovr2'], index=[0])
    
    match['att1'] = wc[wc.name == team1]['att'].iloc[0]
    match['def1'] = wc[wc.name == team1]['def'].iloc[0]
    match['mid1'] = wc[wc.name == team1]['mid'].iloc[0]
    match['ovr1'] = wc[wc.name == team1]['ovr'].iloc[0]

    match['att2'] = wc[wc.name == team2]['att'].iloc[0]
    match['def2'] = wc[wc.name == team2]['def'].iloc[0]
    match['mid2'] = wc[wc.name == team2]['mid'].iloc[0]
    match['ovr2'] = wc[wc.name == team2]['ovr'].iloc[0]
    
    match['att'] = match['att1'] - match['att2']
    match['def'] = match['def1'] - match['def2']
    match['mid'] = match['mid1'] - match['mid2']
    match['ovr'] = match['ovr1'] - match['ovr2']
    
    match = match[['att', 'def', 'mid', 'ovr']]
    
    match_array = match.values
    
    prediction = model.predict(match_array)
    
    winner = None
    
    if prediction == 1:
        winner = team1
    elif prediction == -1:
        winner = team2
    
    return winner
    
    



In [53]:
match(wc, 'brazil', 'spain', svc)

'spain'

In [54]:
def match(wc, team1, team2, model, random_scale=5):
    
    match = pd.DataFrame(columns=['att1','def1','mid1','ovr1','att2','def2','mid2','ovr2'], index=[0])
    
    att1 = wc[wc.name == team1]['att'].iloc[0]
    def1 = wc[wc.name == team1]['def'].iloc[0]
    mid1 = wc[wc.name == team1]['mid'].iloc[0]
    ovr1 = wc[wc.name == team1]['ovr'].iloc[0]

    att2 = wc[wc.name == team2]['att'].iloc[0]
    def2 = wc[wc.name == team2]['def'].iloc[0]
    mid2 = wc[wc.name == team2]['mid'].iloc[0]
    ovr2 = wc[wc.name == team2]['ovr'].iloc[0]
    
    match['att1'] = np.random.normal(att1, scale=random_scale)
    match['def1'] = np.random.normal(def1, scale=random_scale)
    match['mid1'] = np.random.normal(mid1, scale=random_scale)
    match['ovr1'] = np.random.normal(ovr1, scale=random_scale)

    match['att2'] = np.random.normal(att2, scale=random_scale)
    match['def2'] = np.random.normal(def2, scale=random_scale)
    match['mid2'] = np.random.normal(mid2, scale=random_scale)
    match['ovr2'] = np.random.normal(ovr2, scale=random_scale)
    
    match['att'] = match['att1'] - match['att2']
    match['def'] = match['def1'] - match['def2']
    match['mid'] = match['mid1'] - match['mid2']
    match['ovr'] = match['ovr1'] - match['ovr2']
    
    match = match[['att', 'def', 'mid', 'ovr']]
    
    match_array = match.values
    
    prediction = model.predict(match_array)
    
    winner = None
    
    if prediction == 1:
        winner = team1
    elif prediction == -1:
        winner = team2
    
    return winner

In [55]:
def simulate_matches(team1, team2, n_matches=10000):
    
    match_results = []
    for i in range(n_matches):
        match_results.append(match(wc, team1, team2, svc, random_scale=5))
        
    team1_proba = match_results.count(team1) / len(match_results) * 100
    team2_proba = match_results.count(team2) / len(match_results) * 100
    
    print(team1, str(round(team1_proba, 2)) + '%')
    print(team2, str(round(team2_proba,2)) + '%')
    print('-------------------------')
    print()
    
    if team1_proba > team2_proba:
        overall_winner = team1
    else:
        overall_winner = team2
    
    return {'team1': team1,
            'team2': team2,
            'team1_proba': team1_proba, 
            'team2_proba': team2_proba, 
            'overall_winner': overall_winner,
            'match_results': match_results}
    

In [56]:
simulation_test = simulate_matches('croatia', 'denmark', n_matches=10000)

croatia 44.2%
denmark 55.8%
-------------------------



In [57]:
p_list = []
for i in range(len(simulation_test['match_results'])):
    denmark = simulation_test['match_results'][:i].count('denmark') / (i+1) * 100
    croatia = simulation_test['match_results'][:i].count('croatia') / (i+1) * 100
    p_list.append(denmark - croatia)
    

In [58]:
trace = go.Scatter(
    y = p_list,
    name='Teams divergence by number of simulations',
    marker= dict(color='rgb(230,90,110)'))

data = [trace]

py.iplot(data, filename='Divergence')

In [99]:
print('Round of 16:')

ko1 = simulate_matches('uruguay', 'portugal', n_matches=8000)['overall_winner']
ko2 = simulate_matches('france', 'argentina', n_matches=8000)['overall_winner']
ko3 = simulate_matches('brazil', 'mexico', n_matches=8000)['overall_winner']
ko4 = simulate_matches('belgium', 'japan', n_matches=8000)['overall_winner']
ko5 = simulate_matches('spain', 'russia', n_matches=8000)['overall_winner']
ko6 = simulate_matches('croatia', 'denmark', n_matches=8000)['overall_winner']
ko7 = simulate_matches('sweden', 'switzerland', n_matches=8000)['overall_winner']
ko8 = simulate_matches('colombia', 'england', n_matches=8000)['overall_winner']

print()
print('Quarter Finals:')
print()

quarters1 = simulate_matches(ko1, ko2, n_matches=8000)['overall_winner']
quarters2 = simulate_matches(ko3, ko4, n_matches=8000)['overall_winner']
quarters3 = simulate_matches(ko5, ko6, n_matches=8000)['overall_winner']
quarters4 = simulate_matches(ko7, ko8, n_matches=8000)['overall_winner']

print()
print('Semi Finals:')
print()

semifinals1 = simulate_matches(quarters1, quarters2, n_matches=8000)['overall_winner']
semifinals2 = simulate_matches(quarters3, quarters4, n_matches=8000)['overall_winner']

print()
print('Finals:')
print()

finals = simulate_matches(semifinals1, semifinals2, n_matches=8000)


Round of 16:
uruguay 46.96%
portugal 53.04%
-------------------------

france 47.8%
argentina 52.2%
-------------------------

brazil 76.88%
mexico 23.12%
-------------------------

belgium 75.01%
japan 24.99%
-------------------------

spain 75.31%
russia 24.69%
-------------------------

croatia 35.06%
denmark 64.94%
-------------------------

sweden 48.39%
switzerland 51.61%
-------------------------



KeyboardInterrupt: 

In [60]:
spain_proba = 0.817 * 0.7 * 0.862 * 0.7 * 0.718 * 0.7 * 0.593 * 0.7 * 100

print('Chance of Spain winning:', str(round(spain_proba,2)) + '%')

Chance of Spain winning: 7.2%
